In [ ]:
import pandas as pd
from pathlib import Path
import os
import json
from loguru import logger
data_path = Path(Path(os.path.abspath("")).parent, "data")

In [ ]:
%reload_kedro
catalog = context.catalog

In [ ]:
cleaned_mpd_tracks = catalog.load("cleaned_mpd_tracks")

In [ ]:
cleaned_mpd_tracks.head(5)

In [ ]:
count = cleaned_mpd_tracks["track_id"].count().compute()

In [ ]:
count

In [ ]:
import dask.dataframe as dd
import dask.distributed as d_dist
from dask.dataframe.core import repartition
from dask.distributed import Client

In [ ]:
def dask_dedup(ddf, subset):
    client = d_dist.client._get_global_client() or Client()
#     logger.info("Repartitioning")
#     ddf = ddf.repartition(partition_size="100MB")
    logger.info("Deduplicating")
    deduped_ddf = ddf.drop_duplicates(
        subset=subset)
#     deduped_ddf = deduped_cleaned_mpd_tracks.repartition(
#         partition_size="100MB")
    
    return deduped_ddf

In [ ]:
deduped_cleaned_mpd_tracks = dask_dedup(cleaned_mpd_tracks, ["track_id"])

In [ ]:
catalog.save("deduped_cleaned_mpd_tracks", deduped_cleaned_mpd_tracks)